In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [2]:
path = "./win_030103/"
filesName = os.listdir(path)
outputs = []
for line in filesName:
    with open(path+line,encoding="utf-8") as f:
        temp = f.readlines()
        outputs.append(temp)

In [3]:
result = []
iterInfo = []
solInfo = []
index = -1
for line in outputs:
    for row in line:
        row = row.strip()
        if row=="":
            continue
        if "Now" in row and "Sol" not in row:
            index = 1
            result.append([])
        if "iter|" in row:
            index = 2
            iterInfo.append([])
        if "Now" in row and "Sol" in row:
            index = 3
            solInfo.append([])
        if index == 1:
            result[-1].append(row)
        elif index == 2:
            iterInfo[-1].append(row)
        elif index == 3:
            solInfo[-1].append(row)

In [4]:
result_col = []
for row in result[0]:
    temp = row.split(":")
    result_col.append(temp[0].strip())
    
res = pd.DataFrame(columns=result_col)
for index in range(len(result)):
    res_list = []
    for line in result[index]:
        if "Now" in line:
            tempdata = line.split("is")[-1].strip()[1:].strip()
            res_list.append(tempdata)
            continue
        temp = line.split(":")
        tempdata = temp[-1].strip()
        if "txt" in tempdata or "%" in tempdata or "!!" in tempdata:
            res_list.append(tempdata)
        elif "." in tempdata:
            res_list.append(float(tempdata))
        else:
            res_list.append(int(tempdata))
    res.loc[index] = res_list

In [5]:
iterInfo_col = ["seed","Now is","ins"]
temp = iterInfo[0][0].split("|")
for col in temp:
    col = col.strip()
    if col!="":
        iterInfo_col.append(col)
        
ite = pd.DataFrame(columns=iterInfo_col)
n = 0
for index in range(len(iterInfo)):
    if index%100==1:
        print(index,n)
    ite_list_pre = []
    ite_list_pre.append(res.loc[index,"Parameter    seed"])
    ite_list_pre.append(res.loc[index,"Now is"])
    ite_list_pre.append(res.loc[index,"InstanceInfo Ins_Name"])
    for line in iterInfo[index][1:-1]:
        ite_list = []
        temp = line.split("|")
        for tempdata in temp[0:-1]:
            tempdata = tempdata.strip()
            if "." in tempdata:
                ite_list.append(float(tempdata))
            else:
                ite_list.append(int(tempdata))
        ite_list = ite_list_pre + ite_list
        ite.loc[n] = ite_list
        n += 1

1 54
101 6026
201 11917
301 17732
401 23800
501 29765


In [6]:
# solInfo_col = ["seed","Now is","ins","obj","sol"]

# sol = pd.DataFrame(columns=solInfo_col)
# n = 0 
# for index in range(len(solInfo)):
#     if index%100==1:
#         print(index,n)
#     time = solInfo[index][0].split("is")[-1].strip()
#     time = time[1:].strip()
#     sol_list_pre = []
#     sol_list_pre.append(res.loc[index,"Parameter    seed"])
#     sol_list_pre.append(time)
#     for line in solInfo[index]:
#         if "txt" in line:
#             sol_list = []
#             temp = line.split(" ")
#             name = temp[0].strip()
#             obj = temp[1].strip()
#             sss = temp[2].strip()
#             sol_list.append(name)
#             sol_list.append(int(obj))
#             sol_list.append(sss)
#             sol_list = sol_list_pre+sol_list
#             sol.loc[n] = sol_list
#             n += 1

In [7]:
## 汇总数据
instances = []
for line in res.iloc[:,1]:
    if line not in instances:
        instances.append(line)
ins = pd.read_excel("./best.xlsx",index_col=0)
data3 = res[res.loc[:,"SolutionInfo feaNum"]==0]
data3 = data3[data3.loc[:,"SolutionInfo obj"]!=0]
analysis = pd.DataFrame(columns=["n","um","lm","#Best","TotalTime_avg","FindBestTime_avg","favg","fBest","num","Dev","Delta"])
for instance in instances:
    inf = []
    temp = data3[data3.loc[:,"InstanceInfo Ins_Name"]==instance]
    if len(temp)==0:continue
#     inf.append(instance)
    inf.append(temp.iloc[0,1])
    inf.append(temp.iloc[0,3])
    inf.append(temp.iloc[0,4])
    inf.append(ins.bestKown[ins.index==instance][0])
    inf.append(np.average(temp.loc[:,"OutputTime   totalTime"]))
    inf.append(np.average(temp.loc[:,"OutputTime   findBestTime"]))
    inf.append(np.average(temp.loc[:,"SolutionInfo obj"]))
    inf.append(np.max(temp.loc[:,"SolutionInfo obj"]))
    inf.append(len(temp))
    inf.append(np.std(temp.loc[:,"SolutionInfo obj"]))
    inf.append(inf[3]-inf[7])
    analysis.loc[instance]=inf
analysis

C:\Users\gaoming131\AppData\Local\Temp\ipykernel_9964\3297036628.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inf.append(ins.bestKown[ins.index==instance][0])
C:\Users\gaoming131\AppData\Local\Temp\ipykernel_9964\3297036628.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inf.append(ins.bestKown[ins.index==instance][0])
C:\Users\gaoming131\AppData\Local\Temp\ipykernel_9964\3297036628.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `s

n  um  lm   #Best  TotalTime_avg  \
250-10-10-0-0.txt  250-10-10-0-0.txt  10  10   52442      57.858400   
250-10-10-0-1.txt  250-10-10-0-1.txt  10  10   53745      50.313533   
250-10-10-0-2.txt  250-10-10-0-2.txt  10  10   46927      57.778600   
250-10-10-0-3.txt  250-10-10-0-3.txt  10  10   54856      55.575214   
250-10-10-0-4.txt  250-10-10-0-4.txt  10  10   49699      62.701615   
250-10-10-0-5.txt  250-10-10-0-5.txt  10  10   93006      58.174000   
250-10-10-1-0.txt  250-10-10-1-0.txt  10  10   26696      47.262083   
250-10-10-1-1.txt  250-10-10-1-1.txt  10  10   25876      52.156250   
250-10-10-1-2.txt  250-10-10-1-2.txt  10  10   26517      54.573833   
250-10-10-1-3.txt  250-10-10-1-3.txt  10  10   26684      60.510833   
250-10-10-1-4.txt  250-10-10-1-4.txt  10  10   26676      54.653417   
250-10-10-1-5.txt  250-10-10-1-5.txt  10  10   42629      61.376667   
250-10-5-0-0.txt    250-10-5-0-0.txt  10   5   56327      51.475083   
250-10-5-0-1.txt    250-10-5-0-1.txt  10   5   59619      50.114833   
250-10-5-0-2.txt    250-10-5-0-2.txt  10   5   55034      57.321750   
250-10-5-0-3.txt    250-10-5-0-3.txt  10   5   52416      57.907583   
250-10-5-0-4.txt    250-10-5-0-4.txt  10   5   58234      60.136833   
250-10-5-0-5.txt    250-10-5-0-5.txt  10   5   99752      59.315583   
250-10-5-1-0.txt    250-10-5-1-0.txt  10   5   26976      55.699083   
250-10-5-1-1.txt    250-10-5-1-1.txt  10   5   26684      53.475250   
250-10-5-1-2.txt    250-10-5-1-2.txt  10   5   25749      55.204000   
250-10-5-1-3.txt    250-10-5-1-3.txt  10   5   27181      55.662167   
250-10-5-1-4.txt    250-10-5-1-4.txt  10   5   26856      55.571167   
250-10-5-1-5.txt    250-10-5-1-5.txt  10   5   46244      49.816167   
250-5-2-0-0.txt      250-5-2-0-0.txt   5   2   78486      37.269667   
250-5-2-0-1.txt      250-5-2-0-1.txt   5   2   75132      31.683167   
250-5-2-0-2.txt      250-5-2-0-2.txt   5   2   71003      37.583917   
250-5-2-0-3.txt      250-5-2-0-3.txt   5   2   80311      34.251833   
250-5-2-0-4.txt      250-5-2-0-4.txt   5   2   70935      32.774583   
250-5-2-0-5.txt      250-5-2-0-5.txt   5   2  130981      33.876250   
250-5-2-1-0.txt      250-5-2-1-0.txt   5   2   26666      40.116333   
250-5-2-1-1.txt      250-5-2-1-1.txt   5   2   26864      34.762833   
250-5-2-1-2.txt      250-5-2-1-2.txt   5   2   27280      34.762333   
250-5-2-1-3.txt      250-5-2-1-3.txt   5   2   26269      43.479333   
250-5-2-1-4.txt      250-5-2-1-4.txt   5   2   27293      42.411083   
250-5-2-1-5.txt      250-5-2-1-5.txt   5   2   44419      44.491667   
250-5-5-0-0.txt      250-5-5-0-0.txt   5   5   68026      29.131167   
250-5-5-0-1.txt      250-5-5-0-1.txt   5   5   60795      37.589750   
250-5-5-0-2.txt      250-5-5-0-2.txt   5   5   62093      32.117583   
250-5-5-0-3.txt      250-5-5-0-3.txt   5   5   66657      31.753667   
250-5-5-0-4.txt      250-5-5-0-4.txt   5   5   61929      30.382833   
250-5-5-0-5.txt      250-5-5-0-5.txt   5   5  127934      42.547083   
250-5-5-1-0.txt      250-5-5-1-0.txt   5   5   26973      46.962917   
250-5-5-1-1.txt      250-5-5-1-1.txt   5   5   26665      34.919333   
250-5-5-1-2.txt      250-5-5-1-2.txt   5   5   26648      46.218750   
250-5-5-1-3.txt      250-5-5-1-3.txt   5   5   25923      46.763667   
250-5-5-1-4.txt      250-5-5-1-4.txt   5   5   26060      43.561333   
250-5-5-1-5.txt      250-5-5-1-5.txt   5   5   41372      46.944583   

                   FindBestTime_avg           favg   fBest  num        Dev  \
250-10-10-0-0.txt         16.573933   52357.866667   52403   15  32.528688   
250-10-10-0-1.txt          6.092400   53647.200000   53686   15  22.955609   
250-10-10-0-2.txt         10.409400   46804.800000   46927   15  49.742269   
250-10-10-0-3.txt         13.558500   54750.142857   54782   14  35.261993   
250-10-10-0-4.txt         15.654923   49601.153846   49675   13  45.154174   
250-10-10-0-5.txt         10.685615   92920.615385   92967   13  28.342788   
250-10-10

In [8]:
ana = pd.read_excel("./评价体系/ana_0127.xlsx",index_col=0)
def evalute(res):
    # res 中按照顺序存储 ins、obj、fea
    score = ana.loc[res[0],"FeaRate"] / 2
    if res[-1]==0:
        temp = res[1] - ana.loc[res[0],"BestKown"]
        if ana.loc[res[0],"Avg_Delta"]!=ana.loc[res[0],"Upper_Delta"]:
            if temp > ana.loc[res[0],"Avg_Delta"]:
                subS = (temp - ana.loc[res[0],"Avg_Delta"]) / (ana.loc[res[0],"Upper_Delta"] - ana.loc[res[0],"Avg_Delta"])
            elif temp <= ana.loc[res[0],"Avg_Delta"]:
                subS = (temp - ana.loc[res[0],"Avg_Delta"]) / (ana.loc[res[0],"Avg_Delta"] - ana.loc[res[0],"Low_Delta"])
        else:
            if temp > ana.loc[res[0],"Avg_Delta"]:
                subS = 1.0
            else:
                subS = 0.0
        score += ana.loc[res[0],"FeaRate"] / 2 + (1.0 - ana.loc[res[0],"FeaRate"]) * (0.5 + subS*0.5)
    return score
scores = []
for line in res.index:
    temp = []
    temp.append(res.loc[line,"InstanceInfo Ins_Name"])
    temp.append(res.loc[line,"SolutionInfo obj"])
    temp.append(res.loc[line,"SolutionInfo feaNum"])
    score = evalute(temp)
    scores.append(score)
res["score"] = scores

In [9]:
res

Now is InstanceInfo Ins_Name  InstanceInfo Item_Num  \
0    2024-03-01-18:51:29     250-10-10-0-0.txt                    250   
1    2024-03-01-18:50:22     250-10-10-0-0.txt                    250   
2    2024-03-01-19:28:11     250-10-10-0-0.txt                    250   
3    2024-03-01-18:10:51     250-10-10-0-0.txt                    250   
4    2024-03-01-20:05:06     250-10-10-0-0.txt                    250   
..                   ...                   ...                    ...   
584  2024-03-01-18:11:21       250-5-5-1-5.txt                    250   
585  2024-03-01-18:11:53       250-5-5-1-5.txt                    250   
586  2024-03-01-18:50:38       250-5-5-1-5.txt                    250   
587  2024-03-01-18:10:02       250-5-5-1-5.txt                    250   
588  2024-03-01-18:49:28       250-5-5-1-5.txt                    250   

     InstanceInfo Upper_Limit_Num  InstanceInfo Floor_Limit_Num  \
0                              10                            10   
1                              10                            10   
2                              10                            10   
3                              10                            10   
4                              10                            10   
..                            ...                           ...   
584                             5                             5   
585                             5                             5   
586                             5                             5   
587                             5                             5   
588                             5                             5   

     Termination  MaxIter  Termination  timeLimit  Termination  improveCutAdd  \
0                       0                  7.6623                           0   
1                       0                 21.7580                           0   
2                       0                 17.4680                           0   
3                       0                  7.8303                           0   
4                       0                  0.7610                           0   
..                    ...                     ...                         ...   
584                     0                  9.9246                           0   
585                     0                  6.9561                           0   
586                     0                 14.4140                           0   
587                     0                  9.3885                           0   
588                     0                  9.6174                           0   

     Termination  subImproveCut  Termination  subTimeLimit  ...  \
0                          5000                        1.0  ...   
1                          5000                        1.0  ...   
2                          5000                        1.0  ...   
3                          5000                        1.0  ...   
4                          5000                        1.0  ...   
..                          ...                        ...  ...   
584                        5000                        1.0  ...   
585                        5000                        1.0  ...   
586                        5000                        1.0  ...   
587                        5000                        1.0  ...   
588                        5000                        1.0  ...   

     OutputTime   UpdateTime  OutputTime   findBestTime  OutputInfo   iter  \
0                  0.000000%                      2.581                 54   
1                  0.000000%                      2.146                 53   
2                  0.000000%                      9.762                 60   
3                  0.000000%                      7.296                 59   
4                  0.000000%                     53.026                 76   
..                       ...                        ...                ...   
584         

In [11]:
analysis.to_excel(path+"ana.xlsx")
# res.to_excel(path+"result.xlsx")
res.to_excel(path+"score.xlsx")
ite.to_excel(path+"iterInfo.xlsx")
# sol.to_excel(path+"solINfo.xlsx")